In [1]:
import ccxt
import pandas as pd 
import numpy as np
import plotly.graph_objects as go
from datetime import datetime
import collections
import time

//anaconda3/lib/python3.7/site-packages/pandas/compat/_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)


# 1. Consumir datos de CCXT

In [2]:
cryptos=['BTC/USDT','BTC/USDC','ETH/USDT']
limite=20
exchanges=[ccxt.bitmart(),ccxt.bitforex(),ccxt.bibox()]

In [32]:
def order_book(crypto,exchanges):
    limite=20
    ob1,ob2,ob3=[],[],[]
    delay = 60 # seconds
    now=60
    sec=now-60
    while sec<now:
        ob1+=collections.ChainMap(exchanges[0].fetch_order_book(crypto,limit=limite)).maps
        ob2+=collections.ChainMap(exchanges[1].fetch_order_book(crypto,limit=limite)).maps
        ob3+=collections.ChainMap(exchanges[2].fetch_order_book(crypto,limit=limite)).maps
        time.sleep (delay)
        sec+=1
    return ob1,ob2,ob3

In [107]:
ob_c1=order_book(cryptos[0],exchanges)

In [5]:
ob_c2=order_book(cryptos[1],exchanges)

In [6]:
ob_c3=order_book(cryptos[2],exchanges)

In [70]:
def times(exchanges,data):
    exchange,times=[],[]
    for i in range(len(exchanges)):
        exchange.append([str(exchanges[i])]*seconds)
        for j in range(len(data[0])):
            times.append(data[i][j]['datetime'])
    return times

In [71]:
def close_prices(exchanges,symbol,data,n):
    time=times(exchanges,data)[60*n]
    startDate=exchanges[n].parse8601(time)
    ohlcv = exchanges[n].fetch_ohlcv(symbol, timeframe='1m', since=startDate,limit=60)
    closes = [x[4] for x in ohlcv]
    dates = [exchanges[n].iso8601 (x[0]) for x in ohlcv]
    return closes,dates

In [73]:
c1m1=close_prices(exchanges,cryptos[0],ob_c1,0)
c1m2=close_prices(exchanges,cryptos[0],ob_c1,1)
c1m3=close_prices(exchanges,cryptos[0],ob_c1,2)

c2m1=close_prices(exchanges,cryptos[1],ob_c2,0)
c2m2=close_prices(exchanges,cryptos[1],ob_c2,1)
c2m3=close_prices(exchanges,cryptos[1],ob_c2,2)

c3m1=close_prices(exchanges,cryptos[2],ob_c3,0)
c3m2=close_prices(exchanges,cryptos[2],ob_c3,1)
c3m3=close_prices(exchanges,cryptos[2],ob_c3,2)

## 1.2 Verificacion de avance

In [172]:
d={'Exchange_1 '+str(exchanges[0]):{'Crypto1':ob_c1[0],
                 'Closes':c1m1[0],
                 'Crypto2':ob_c2[0],
                 'Closes':c2m1[0],
                 'Crypto3':ob_c3[0],
                 'Closes':c3m1[0]},
  'Exchange_2 '+str(exchanges[1]):{'Crypto1':ob_c1[1],
                 'Closes':c1m2[0],
                 'Crypto2':ob_c2[1],
                 'Closes':c2m2[0],
                 'Crypto3':ob_c3[1],
                 'Closes':c3m2[0]},
  'Exchange_3 '+str(exchanges[2]):{'Crypto1':ob_c1[2],
                 'Closes':c1m3[0],
                 'Crypto2':ob_c2[2],
                 'Closes':c2m3[0],
                 'Crypto3':ob_c3[2],
                 'Closes':c3m3[0]}}

In [173]:
d

{'Exchange_1 BitMart': {'Crypto1': [{'symbol': 'BTC/USDT',
    'bids': [[17987.96, 0.08478],
     [17987.85, 0.29146],
     [17987.75, 0.11742],
     [17987.6, 0.1099],
     [17987.49, 0.15987],
     [17987.2, 0.28237],
     [17986.87, 0.0952],
     [17986.74, 0.23779],
     [17986.55, 0.2257],
     [17986.45, 0.07799],
     [17986.34, 0.29001],
     [17986.23, 0.18082],
     [17986.12, 0.1671],
     [17986.01, 0.29218],
     [17985.9, 0.24628],
     [17985.7, 0.2009],
     [17985.59, 0.24339],
     [17985.48, 0.24286],
     [17985.26, 0.29116],
     [17984.94, 0.11776]],
    'asks': [[17988.7, 0.04655],
     [17989.19, 0.00325],
     [17990.3, 0.19509],
     [17990.4, 0.29292],
     [17990.51, 0.21691],
     [17990.62, 0.25168],
     [17990.73, 0.21932],
     [17990.84, 0.29385],
     [17990.94, 0.12072],
     [17991.16, 0.21179],
     [17991.27, 0.19354],
     [17991.38, 0.16467],
     [17991.48, 0.28335],
     [17991.59, 0.28898],
     [17991.62, 0.25486],
     [17991.73, 0.28094],


In [174]:
d['Exchange_1 BitMart']['Crypto2']

[{'symbol': 'BTC/USDC',
  'bids': [[17707.38, 0.003427],
   [17706.75, 0.048169],
   [17706.65, 0.017378],
   [17706.54, 0.040058],
   [17705.69, 0.043461],
   [17705.58, 0.01579],
   [17705.37, 0.049341],
   [17704.98, 0.028214],
   [17704.95, 0.011923],
   [17703.99, 0.03849],
   [17701.97, 0.039951],
   [17701.7, 0.031543],
   [17701.49, 0.020061],
   [17701.25, 0.047498],
   [17701.14, 0.049665],
   [17697.51, 0.89861],
   [17693.49, 0.951278],
   [17691.15, 0.581968],
   [17688.71, 0.790573],
   [17685.16, 0.824507]],
  'asks': [[17708.25, 0.010478],
   [17708.66, 0.048107],
   [17708.98, 0.049834],
   [17709.19, 0.032208],
   [17709.3, 0.033765],
   [17709.7, 0.050322],
   [17709.81, 0.038703],
   [17709.91, 0.045253],
   [17710.02, 0.035046],
   [17710.13, 0.017101],
   [17710.23, 0.03291],
   [17710.34, 0.028032],
   [17710.55, 0.012494],
   [17710.76, 0.014993],
   [17710.87, 0.04958],
   [17714.19, 0.779308],
   [17717.92, 0.774498],
   [17721.09, 0.568221],
   [17723.76, 0.8

# 2. Visualizacion de microestructura

## 2.2 Verificación de avance

In [91]:
def verifavance2(data,limite,seconds,exchanges):
    exchange,times=[],[]
    ask_vol,bid_vol=[],[]
    ask_voll,bid_voll=[],[]
    mid,spread=[],[]
    ask,bid=[],[]
    c=0
    df=pd.DataFrame(columns=['exchange','timeStamp','level','ask_volume','bid_volume','total_volume','mid_price','vwap'])
    for i in range(len(exchanges)):
        exchange.append([str(exchanges[i])]*seconds)
        for j in range(seconds):
            times.append(data[i][j]['datetime'])
            mid.append((data[i][j]['bids'][0][0]+data[i][j]['asks'][0][0])/2)
            for m in range(limite):
                ask_vol.append(data[i][j]['asks'][m][1])
                bid_vol.append(data[i][j]['bids'][m][1])
                ask.append(data[i][j]['asks'][m][0])
                bid.append(data[i][j]['bids'][m][0])
            ask_voll.append(sum(ask_vol[c:c+limite]))
            bid_voll.append(sum(bid_vol[c:c+limite]))
            spread.append(np.mean(ask[c:c+limite])-np.mean(bid[c:c+limite]))
            c+=limite
    exchange=[exchange for exchanges in exchange for exchange in exchanges]
    df['exchange']=exchange
    df['timeStamp']=times
    df['level']=limite
    df['ask_volume']=ask_voll
    df['bid_volume']=bid_voll
    df['total_volume']=df['ask_volume']+df['bid_volume']
    df['mid_price']=mid
    df['vwap']=sum(spread*df['total_volume'])/df['total_volume']
    return df, spread

In [92]:
vac1=verifavance2(ob_c1,limite,60,exchanges)
vac2=verifavance2(ob_c2,limite,60,exchanges)
vac3=verifavance2(ob_c3,limite,60,exchanges)

In [94]:
vac1[0]

,exchange,timeStamp,level,ask_volume,bid_volume,total_volume,mid_price,vwap
0,BitMart,2022-11-10T21:08:41.923Z,20,4.002680,3.954940,7.957620,17988.330,6.287418e+06
1,BitMart,2022-11-10T21:09:43.290Z,20,20.222350,3.369440,23.591790,17940.410,2.120775e+06
2,BitMart,2022-11-10T21:10:46.067Z,20,33.843000,6.295630,40.138630,17956.590,1.246502e+06
3,BitMart,2022-11-10T21:11:48.999Z,20,27.817640,23.399530,51.217170,17968.270,9.768771e+05
4,BitMart,2022-11-10T21:12:51.551Z,20,29.111830,25.717280,54.829110,17984.775,9.125241e+05
...,...,...,...,...,...,...,...,...
175,Bibox,2022-11-10T21:59:59.786Z,20,15.266126,18.665799,33.931925,17871.650,1.474508e+06
176,Bibox,2022-11-10T21:59:59.786Z,20,15.266126,18.665799,33.931925,17871.650,1.474508e+06
177,Bibox,2022-11-10T22:07:32.102Z,20,13.873980,13.891096,27.765076,17806.350,1.802008e+06
178,Bibox,2022-11-10T22:08:33.228Z,20,15.246435,19.132296,34.378731,17752.100,1.455344e+06


In [110]:
vac2[0]

,exchange,timeStamp,level,ask_volume,bid_volume,total_volume,mid_price,vwap
0,BitMart,2022-11-10T22:10:35.161Z,20,4.147298,4.531905,8.679203,17707.815,4.796208e+06
1,BitMart,2022-11-10T22:11:36.310Z,20,0.613528,0.590914,1.204442,17682.250,3.456145e+07
2,BitMart,2022-11-10T22:12:38.359Z,20,14.772824,8.379592,23.152416,17624.925,1.797966e+06
3,BitMart,2022-11-10T22:13:42.554Z,20,5.263581,10.534632,15.798213,17675.795,2.634935e+06
4,BitMart,2022-11-10T22:14:43.658Z,20,12.606572,4.979203,17.585775,17639.760,2.367098e+06
...,...,...,...,...,...,...,...,...
175,Bibox,2022-11-10T23:05:05.000Z,20,3.203900,545.562800,548.766700,17795.325,7.585602e+04
176,Bibox,2022-11-10T23:08:11.833Z,20,3.801400,942.525700,946.327100,17811.335,4.398824e+04
177,Bibox,2022-11-10T23:09:13.000Z,20,2.861300,916.150500,919.011800,17812.235,4.529567e+04
178,Bibox,2022-11-10T23:10:15.000Z,20,1.097900,763.797000,764.894900,17815.125,5.442219e+04


In [111]:
vac3[0]

,exchange,timeStamp,level,ask_volume,bid_volume,total_volume,mid_price,vwap
0,BitMart,2022-11-10T23:12:17.900Z,20,9.90650,8.23125,18.13775,1325.155,4803.636536
1,BitMart,2022-11-10T23:13:20.394Z,20,36.73056,161.94637,198.67693,1323.630,438.536868
2,BitMart,2022-11-10T23:14:21.516Z,20,127.48985,180.57730,308.06715,1319.440,282.818725
3,BitMart,2022-11-10T23:15:22.774Z,20,188.33963,101.78361,290.12324,1316.680,300.310856
4,BitMart,2022-11-10T23:16:23.759Z,20,256.17878,310.49736,566.67614,1307.455,153.751239
...,...,...,...,...,...,...,...,...
175,Bibox,2022-11-11T00:08:09.791Z,20,191.07330,181.48060,372.55390,1305.815,233.864573
176,Bibox,2022-11-11T00:09:51.056Z,20,188.14720,258.30650,446.45370,1303.770,195.153850
177,Bibox,2022-11-11T00:10:53.582Z,20,183.93010,204.46320,388.39330,1302.110,224.327141
178,Bibox,2022-11-11T00:11:55.402Z,20,256.90780,237.22030,494.12810,1291.370,176.325043


## 2.4 Visualizaciones

In [78]:
def visualization(data,crypto):
    d1=data[data['exchange']==str(exchanges[0])]
    d2=data[data['exchange']==str(exchanges[1])]
    d3=data[data['exchange']==str(exchanges[2])]
    fig = go.Figure(layout=dict(title=dict(text=crypto+" Mid Price")))
    fig.add_trace(go.Scatter(x=d1['timeStamp'], y=d1['mid_price'],
                        mode='lines',name=str(exchanges[0])+' '+crypto+': MidPrice',line=dict(color='skyblue')))
    fig.add_trace(go.Scatter(x=d2['timeStamp'], y=d2['mid_price'],
                        mode='lines',name=str(exchanges[1])+' '+crypto+': MidPrice',line=dict(color='lightsalmon')))
    fig.add_trace(go.Scatter(x=d3['timeStamp'], y=d3['mid_price'],
                        mode='lines',name=str(exchanges[2])+' '+crypto+': MidPrice',line=dict(color='lawngreen')))
    fig.update_xaxes(
            title_text = "TimeStamp",
            title_standoff = 25)

    fig.update_yaxes(
            title_text = "MidPrice",
            title_standoff = 25)
    return fig.show()

In [95]:
visualization(vac1[0],cryptos[0])

In [96]:
visualization(vac2[0],cryptos[1])

In [97]:
visualization(vac3[0],cryptos[2])

# 3. Modelado de microestructura

## 3.2 Verificacion de avance

In [19]:
closec1=c1m1[0]+c1m2[0]+c1m3[0]
closec2=c2m1[0]+c2m2[0]+c2m3[0]
closec3=c3m1[0]+c3m2[0]+c3m3[0]

date_closec1=c1m1[1]+c1m2[1]+c1m3[1]
date_closec1=c2m1[1]+c2m2[1]+c2m3[1]
date_closec1=c3m1[1]+c3m2[1]+c3m3[1]

In [101]:
def verifavance3(spread):
    df=pd.DataFrame(columns=['timestamp','Close','Spread','Effective Spread'])
    df['timestamp']=c1m1[1]+c1m2[1]+c1m3[1]
    df['Close']=c1m1[0]+c1m2[0]+c1m3[0]
    df['Spread']=spread
    for i in range(len(df)-5):
        df.loc[i+5,'Effective Spread']=2*np.sqrt(np.abs(np.cov(np.diff(df.loc[0:i+5,'Close']))))
    return df

In [102]:
va3_1=verifavance3(vac1[1])
va3_1

,timestamp,Close,Spread,Effective Spread
0,2022-11-10T21:09:00.000Z,17930.32,4.5390,NaN
1,2022-11-10T21:10:00.000Z,17961.57,11.1295,NaN
2,2022-11-10T21:11:00.000Z,17975.22,18.9165,NaN
3,2022-11-10T21:12:00.000Z,17980.58,18.9735,NaN
4,2022-11-10T21:13:00.000Z,17988.17,30.5000,NaN
...,...,...,...,...
175,2022-11-10T22:04:00.000Z,17844.70,8.7050,74.565079
176,2022-11-10T22:05:00.000Z,17818.30,8.7050,74.454269
177,2022-11-10T22:06:00.000Z,17825.10,6.8350,74.250866
178,2022-11-10T22:07:00.000Z,17763.70,9.6000,74.599791


In [108]:
va3_2=verifavance3(vac2[1])
va3_2

,timestamp,Close,Spread,Effective Spread
0,2022-11-10T21:09:00.000Z,17930.32,11.4605,NaN
1,2022-11-10T21:10:00.000Z,17961.57,2.9305,NaN
2,2022-11-10T21:11:00.000Z,17975.22,38.8990,NaN
3,2022-11-10T21:12:00.000Z,17980.58,26.1560,NaN
4,2022-11-10T21:13:00.000Z,17988.17,31.7450,NaN
...,...,...,...,...
175,2022-11-10T22:04:00.000Z,17844.70,121.2495,74.565079
176,2022-11-10T22:05:00.000Z,17818.30,127.0850,74.454269
177,2022-11-10T22:06:00.000Z,17825.10,123.9210,74.250866
178,2022-11-10T22:07:00.000Z,17763.70,125.1230,74.599791


In [109]:
va3_3=verifavance3(vac3[1])
va3_3

,timestamp,Close,Spread,Effective Spread
0,2022-11-10T21:09:00.000Z,17930.32,0.2985,NaN
1,2022-11-10T21:10:00.000Z,17961.57,1.1340,NaN
2,2022-11-10T21:11:00.000Z,17975.22,1.3450,NaN
3,2022-11-10T21:12:00.000Z,17980.58,1.0025,NaN
4,2022-11-10T21:13:00.000Z,17988.17,6.2270,NaN
...,...,...,...,...
175,2022-11-10T22:04:00.000Z,17844.70,0.7800,74.565079
176,2022-11-10T22:05:00.000Z,17818.30,0.9975,74.454269
177,2022-11-10T22:06:00.000Z,17825.10,1.0690,74.250866
178,2022-11-10T22:07:00.000Z,17763.70,0.9940,74.599791
